In [1]:
import psycopg2

In [2]:
# Create a connection to the database, get a cursor, and set autocommit to true
try:
 conn = psycopg2.connect("host=127.0.0.1 dbname=demo001")
except psycopg2.Error as e:
 print("Error: Could not make connection to the Postgres database")
 print(e)
try:
 cur = conn.cursor()
except psycopg2.Error as e:
 print("Error: Could not get curser to the Database")
 print(e)
conn.set_session(autocommit=True)

### There will be many tables in our database but let's just focus on 4 tables around customer purchases.

Table Name: customer_transactions
- column: customer_id
- column: store_id
- column: spent

Table Name: customer
- column: customer_id
- column: name
- column: rewards

Table Name: store
- column: store_id
- column: state

Table Name: items_purchased
- column: customer_id
- column: item_name

From this representation we can already start to see the makings of a "STAR". We have one fact table (the center of the star) and 3 dimension tables that are coming from it."

In [3]:
# Let's create the Fact Table and insert the data into the table
try:
 cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions (customer_id int, store_id int, spent numeric);")
except psycopg2.Error as e:
 print("Error: Issue creating table", e)

#Insert into all tables
try:
 cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) \
 VALUES (%s, %s, %s)", \
 (1, 1, 20.50))
except psycopg2.Error as e:
 print("Error: Inserting Rows", e)
try:
 cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) \
 VALUES (%s, %s, %s)", \
 (2, 1, 35.21))
except psycopg2.Error as e:
 print("Error: Inserting Rows", e)

In [4]:
# Let's create our Dimension Tables and insert data into those tables.
try:
 cur.execute("CREATE TABLE IF NOT EXISTS items_purchased (customer_id int, item_number int, item_name varchar);")
except psycopg2.Error as e:
 print("Error: Issue creating table")
 print (e)

try:
 cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) \
 VALUES (%s, %s, %s)", \
 (1, 1, "Rubber Soul"))
except psycopg2.Error as e:
 print("Error: Inserting Rows")
 print (e)

try:
 cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) \
 VALUES (%s, %s, %s)", \
 (2, 3, "Let It Be"))
except psycopg2.Error as e:
 print("Error: Inserting Rows")
 print (e)

try:
 cur.execute("CREATE TABLE IF NOT EXISTS store (store_id int, state varchar);")
except psycopg2.Error as e:
 print("Error: Issue creating table")
 print (e)

try:
 cur.execute("INSERT INTO store (store_id, state) \
 VALUES (%s, %s)", \
 (1, "CA"))
except psycopg2.Error as e:
 print("Error: Inserting Rows")
 print (e)
try:
 cur.execute("INSERT INTO store (store_id, state) \
 VALUES (%s, %s)", \
 (2, "WA"))
except psycopg2.Error as e:
 print("Error: Inserting Rows")
 print (e)

try:
 cur.execute("CREATE TABLE IF NOT EXISTS customer (customer_id int, name varchar, rewards boolean);")
except psycopg2.Error as e:
 print("Error: Issue creating table")
 print (e)

try:
 cur.execute("INSERT INTO customer (customer_id, name, rewards) \
 VALUES (%s, %s, %s)", \
 (1, "Amanda", True))
except psycopg2.Error as e:
 print("Error: Inserting Rows")
 print (e)

try:
 cur.execute("INSERT INTO customer (customer_id, name, rewards) \
 VALUES (%s, %s, %s)", \
 (2, "Toby", False))
except psycopg2.Error as e:
 print("Error: Inserting Rows", e)

### We can do a variety of queries on this data easily because of utilizing the fact/dimension and Star Schema

- Query 1: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member 

- Query 2: How much did Store 1 sell?

In [5]:
# QUERY 1
try:
 cur.execute("SELECT name, item_name, rewards FROM ((customer_transactions \
                JOIN customer ON customer.customer_id=customer_transactions.customer_id)\
                JOIN items_purchased ON customer_transactions.customer_id=items_purchased.customer_id)\
                WHERE spent > 30 ;")


except psycopg2.Error as e:
 print("Error: select *")
 print (e)

row = cur.fetchone()
while row:
 print(row)
 row = cur.fetchone()

('Toby', 'Let It Be', False)


In [6]:
# Query 2:_ How much did Store 1 sell?
try:
 cur.execute("SELECT store_id, SUM(spent) FROM customer_transactions GROUP BY store_id;")

except psycopg2.Error as e:
 print("Error: select *")
 print (e)

row = cur.fetchone()
while row:
 print(row)
 row = cur.fetchone()

(1, Decimal('55.71'))


### Summary: 
From this elegant schema we were able to get "facts/metrics" from our fact table (how much each store sold), and also information about our customers that will allow us to do more indepth analytics to get answers to business questions by utilizing our fact and dimension tables. 

In [7]:
# For demo sake drop the table.
try:
 cur.execute("DROP table customer_transactions")
except psycopg2.Error as e:
 print("Error: Dropping table", e)
try:
 cur.execute("DROP table items_purchased")
except psycopg2.Error as e:
 print("Error: Dropping table")
 print (e)
try:
 cur.execute("DROP table customer")
except psycopg2.Error as e:
 print("Error: Dropping table")
 print (e)
try:
 cur.execute("DROP table store")
except psycopg2.Error as e:
 print("Error: Dropping table")
 print (e)

In [8]:
# And finally close your cursor and connection.
cur.close()
conn.close()